In [12]:
import pandas as pd
import dask.dataframe as dd
import daft
import polars as pl
import pyspark

from daft import col, DataType

In [2]:
json_data = [
    {
        "id": 1,
        "name": "John Doe",
        "age": 30,
        "attribute": {
            "height": 176,
            "weight": 80
        }
    },
    {
        "id": 2,
        "name": "Alice Smith",
        "age": 28,
        "attribute": {
            "height": 167,
            "weight": 55
        }
    },
    {
        "id": 3,
        "name": "Bob Johnson",
        "age": 35,
        "attribute": {
            "height": 192,
            "weight": 85
        }
    }
]

In [3]:
# write to JSON file
import json
with open('data.json', 'w', encoding='utf8') as f:
    json.dump(json_data, f)

In [6]:
# read with pandas
pandas_df = pd.read_json('data.json')
print(pandas_df)

   id         name  age                      attribute
0   1     John Doe   30  {'height': 176, 'weight': 80}
1   2  Alice Smith   28  {'height': 167, 'weight': 55}
2   3  Bob Johnson   35  {'height': 192, 'weight': 85}


In [8]:
# read with daft
daft_df = daft.read_json("data.json")
daft_df.show()

ValueError: DaftError::External External format error: Deserialized JSON value is not an Object record

In [10]:
# get specific values with pandas
json_column = pandas_df['attribute']
extracted_data = json_column.apply(lambda x: x['height'])
print(extracted_data)

0    176
1    167
2    192
Name: attribute, dtype: int64


In [11]:
# get specific values with daft
df_daft = daft.from_pandas(pandas_df)
df_daft.show()

idInt64,nameUtf8,ageInt64,"attributeStruct[height: Int64, weight: Int64]"
1,John Doe,30,"{height: 176,weight: 80,}"
2,Alice Smith,28,"{height: 167,weight: 55,}"
3,Bob Johnson,35,"{height: 192,weight: 85,}"


In [13]:
# attribute col is of type structured, needs to be string
df_daft.with_column("attribute_string", col("attribute").cast(DataType.string())).show()

/Users/rpelgrim/miniforge3/envs/daft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
GlobalLimit-Project [Stage:1]:   0%|                                                              | 0/1 [00:00<?, ?it/s]thread '<unnamed>' panicked at 'not implemented: Daft casting from Struct[height: Int64, weight: Int64] to Utf8 not implemented', src/daft-core/src/array/ops/cast.rs:1792:18
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: not implemented: Daft casting from Struct[height: Int64, weight: Int64] to Utf8 not implemented